In [112]:
import struct
from struct import (
    unpack,
unpack_from,
    calcsize,
)

from dataclasses import make_dataclass

In [4]:
with open('none.3mm', 'rb') as f:
    data = f.read()

In [5]:
data

b'CHN2DEHC\x05\x00\x04\x00\x01\x00\x03\x03\xa9\x01\x00\x00\x19\x01\x00\x00\x90\x00\x00\x00\xa9\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x03\x03 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\x01\x00\x03\x03\x08\x00\x00\x00\x04\x00\x00\x00I\x00\x00\x00\xff\xff\xff\xff\x13This movie document\x0fYour User Rolls\x163D Movie Maker/3DMovie\rPlaydo/Playdo\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00\x01\x00\x00\x00$\x00\x00\x00\x02\x00\x00\x00;\x00\x00\x00\x03\x00\x00\x00\x01\x00\x03\x03\x02\x00\x02\x00\x01\x00\x03\x03\x03\x00\x00\x00d\x00\x00\x00\xff\xff\xff\xff\x14\x00\x00\x00EIVM\x00\x00\x00\x00\x11\x01\x00\x00\x02\x

In [ ]:
# header
# raw chunk data
# index
# optional free map

# struct ChunkyFilePrefix

In [132]:

# // chunky file prefix
# struct ChunkyFilePrefix
# {
#     long lwMagic;   // identifies this as a chunky file
#     ChunkTag ctgCreator; // program that created this file
#     DataVersion dver;      // chunky file version
#     short bo;       // byte order
#     short osk;      // which system wrote this

#     FP fpMac;     // logical end of file
#     FP fpIndex;   // location of chunky index
#     long cbIndex; // size of chunky index
#     FP fpMap;     // location of free space map
#     long cbMap;   // size of free space map (may be 0)

#     long rglwReserved[23]; // reserved for future use - should be zero
# };
# const ByteOrderMask kbomCfp = 0xB55FFC00L;


reserved_size = calcsize('23l')
ChunkyFilePrefix_format = f'<4s4shhhhlllll{reserved_size}s'
ChunkyFilePrefix_size = calcsize(ChunkyFilePrefix_format)

ChunkyFilePrefix = make_dataclass('ChunkyFilePrefix', [
    ('lwMagic', bytes),
    ('ctgCreator', bytes),
    ('dver_swCur', int),
    ('dver_swBack', int),
    ('bo', int),
    ('osk', int),
    ('fpMac', int),
    ('fpIndex', int),
    ('cbIndex', int),
    ('fpMap', int),
    ('cbMap', int),
    ('rglwReserved', bytes),
])

cfp = ChunkyFilePrefix(*unpack_from(ChunkyFilePrefix_format, data, 0))

# struct GroupOnFile

In [130]:
# struct GroupOnFile
# {
#     short bo;
#     short osk;
#     long ilocMac;
#     long bvMac;
#     long clocFree;
#     long cbFixed;
# };

GroupOnFile_format = '<hhllll'
GroupOnFile_size   = calcsize(ggf_format)


GroupOnFile = make_dataclass('GroupOnFile', [
    ('bo', int),
    ('osk', int),
    ('ilocMac', int),
    ('bvMac', int),
    ('clocFree', int),
    ('cbFixed', int),
])

x pad byte

h H short
i I int
l L long
q Q long long
s char[] 10s len(s) == 10
p char[] pascal string
P void*

843991107

(843991107,)

843991107

In [151]:
# /****************************************
#     Byte order issues
# ****************************************/

kboCur  = 0x0001
kboOther = 0x0100

kcvnCur = 5

kcvnBack = 4

kcvnMin = 1

kcvnMinStnNames = 3
kcvnMinGrfcrp = 4
kcvnMinSmallIndex = 4
kcvnMinForest = 5

In [133]:
klwMagicChunky = b'CHN2'
assert cfp.lwMagic == klwMagicChunky

In [40]:
assert cfp.ctgCreator == b'DEHC'

In [140]:
assert cfp.rglwReserved == b'\x00\x00\x00\x00' * 23

In [144]:
    # struct LogicalOffsetAndCount
    # {
    #     long bv;
    #     long cb;
    # };

LogicalOffsetAndCount_format = '<ll'
LogicalOffsetAndCount_size   = calcsize(LogicalOffsetAndCount_format)


LogicalOffsetAndCount = make_dataclass('LogicalOffsetAndCount', [
    ('bv', int),
    ('cb', int),
])

In [ ]:
# /***************************************************************************
#     If the block is packed, unpack it.
# ***************************************************************************/
# bool DataBlock::FUnpackData(void)
# {
#     AssertThis(0);
#     HQ hq;

#     if (!_fPacked)
#         return fTrue;

#     if (pvNil != _flo.pfil)
#     {
#         if (!_flo.FReadHq(&hq))
#             return fFalse;
#         if (!vpcodmUtil->FDecompressPhq(&hq))
#         {
#             FreePhq(&hq);
#             AssertThis(fblckPacked | fblckFile);
#             return fFalse;
#         }
#         SetHq(&hq, fFalse);
#     }
#     else if (hqNil == _hq)
#         return fFalse;
#     else
#     {
#         AssertHq(_hq);
#         if (_ibMin != 0 || _ibLim != CbOfHq(_hq))
#         {
#             hq = HqFree();
#             SetHq(&hq, fTrue);
#         }
#         Assert(_ibMin = 0 && _ibLim == CbOfHq(_hq), 0);
#         if (!vpcodmUtil->FDecompressPhq(&_hq))
#         {
#             AssertThis(fblckPacked | fblckMemory);
#             return fFalse;
#         }
#         _ibMin = 0;
#         _ibLim = CbOfHq(_hq);
#         _fPacked = fFalse;
#     }

#     AssertThis(fblckUnpacked | fblckMemory);
#     return fTrue;
# }

In [ ]:
# /***************************************************************************
#     Swap bytes in pv according to bom.  bom consists of up to 16
#     2-bit opcodes (packed from hi bit to low bit).  The high bit of
#     each opcode indicates a long field (1) or short field (0).  The low
#     bit of each opcode indicates whether the bytes are to be swapped
#     in the field (1) or left alone (0).
# ***************************************************************************/
# void SwapBytesBom(void *pv, ByteOrderMask bom)
# {
#     byte b;
#     byte *pb = (byte *)pv;

#     Assert(size(short) == 2 && size(long) == 4, "code broken");
#     while (bom != 0)
#     {
#         if (bom & 0x80000000L)
#         {
#             // long field
#             AssertPvCb(pb, 4);
#             if (bom & 0x40000000L)
#             {
#                 b = pb[3];
#                 pb[3] = pb[0];
#                 pb[0] = b;
#                 b = pb[2];
#                 pb[2] = pb[1];
#                 pb[1] = b;
#             }
#             pb += 4;
#         }
#         else
#         {
#             // short field
#             AssertPvCb(pb, 2);
#             if (bom & 0x40000000L)
#             {
#                 b = pb[1];
#                 pb[1] = pb[0];
#                 pb[0] = b;
#             }
#             pb += 2;
#         }
#         bom <<= 2;
#     }
# }

In [163]:
# /***************************************************************************
#     Read a range of bytes from the data block.
# ***************************************************************************/
# bool DataBlock::FReadRgb(void *pv, long cb, long ib, bool fPackedOk)
# {

def ReadRangeOfBytes(block, cb, ib, packed_ok=False):
    return block[cb:cb+ib]

    
#     AssertThis(0);
#     AssertPvCb(pv, cb);

#     if (!fPackedOk && _fPacked)
#     {
#         Bug("reading packed data");
#         return fFalse;
#     }

#     if (!_FRangeIn(Cb(fTrue), cb, ib))
#     {
#         Bug("reading outside blck");
#         return fFalse;
#     }

#     if (pvNil != _flo.pfil)
#         return _flo.FReadRgb(pv, cb, ib);

#     if (hqNil != _hq)
#     {
#         CopyPb((byte *)QvFromHq(_hq) + ib + _ibMin, pv, cb);
#         return fTrue;
#     }

#     Assert(cb == 0 && ib == 0, 0);
#     return fTrue;
# }

In [159]:
# /***************************************************************************
#     Read the two sections of data from the given location in the given
#     block.
# ***************************************************************************/
# bool GroupBase::_FReadData(PDataBlock pblck, long cb1, long cb2, long ib)
# {

def ReadDataGroupBase(block, cb1, cb2, ib):
    pass
    
#     AssertPo(pblck, fblckUnpacked);
#     AssertIn(cb1, 0, kcbMax);
#     AssertIn(cb2, 0, kcbMax);



#     if (cb1 == 0 && cb2 == 0)
#         return fTrue;

    if cb1 == 0 and cb2 == 0:
        return None
        
#     if (!_FEnsureSizes(cb1, cb2, fgrpNil))
#         return fFalse;

#     if (cb1 > 0)
#     {
#         fRet = pblck->FReadRgb(PvLockHq(_hqData1), cb1, ib);
#         UnlockHq(_hqData1);
#         if (!fRet)
#             return fFalse;
#     }

    if cb1 > 0:
        data1 = ReadRangeOfBytes(block, cb1, ib)

    if cb2 > 0:
        data2 = ReadRangeOfBytes(block, cb2, ib + cb1)

    return (data1, data2)
# }

In [165]:
# /***************************************************************************
#     Read group data from disk.
# ***************************************************************************/
# bool VirtualGroup::_FRead(PDataBlock pblck, short *pbo, short *posk)
# {

def FRead_VirtualGroup(block, cfp):
    fRet = False

#     if (!pblck->FUnpackData())
#         goto LFail;

    cb = len(block)
    if cb < ggf_size:
        raise ValueError

    ggf = GroupOnFile(*unpack_from(ggf_format, block, 0))

    if cfp.bo == 0:
        cfp.bo = ggf.bo
        
    if cfp.osk != 0:
        cfp.osk = ggf.osk

#     if ((bo = ggf.bo) == kboOther)
#         SwapBytesBom(&ggf, kbomGgf);


    cb -= ggf_size
    
    cbT = ggf.ilocMac * LogicalOffsetAndCount_size
#     if (ggf.bo != kboCur || ggf.bvMac < 0 || ggf.ilocMac < 0 || cb != cbT + ggf.bvMac || ggf.cbFixed < 0 ||
#         ggf.cbFixed >= kcbMax || (ggf.clocFree == cvNil) != (_clocFree == cvNil) ||
#         ggf.clocFree != cvNil && (ggf.clocFree < 0 || ggf.clocFree >= ggf.ilocMac))
#     {
#     LBug:
#         Warn("file corrupt or not a VirtualGroup");
#         goto LFail;
#     }

#     _ivMac = ggf.ilocMac;
#     _bvMac = ggf.bvMac;
#     _clocFree = ggf.clocFree;
#     _cbFixed = ggf.cbFixed;

#     fRet = _FReadData(pblck, cb - cbT, cbT, size(ggf));
    Group = ReadDataGroupBase(block, cb - cbT, cbT, ggf_size)
    return Group

#     AssertBomRglw(kbomLoc, size(LOC));
#     if (bo == kboOther && fRet)
#     {
#         // adjust the byte order on the loc's.
#         SwapBytesRglw(_Qb2(0), LwMulDiv(_ivMac, size(LOC), size(long)));
#     }

# LFail:
#     TrashVarIf(!fRet, pbo);
#     TrashVarIf(!fRet, posk);
#     return fRet;
# }




block = data[cfp.fpIndex:cfp.fpIndex+cfp.cbIndex]
FRead_VirtualGroup(block, cfp)

(b' TSG\x03\x00\x00\x00\x94\x00\x00\x00\x00}\x00\x00\x00\x00\x01\x00',
 b'\x00\x00\x00\x00\x11\x01\x00\x00\x02\x08\x00\x00\x02\x00\x00\x00 TSG\x02\x00\x00\x00\x00\x00\x00\x00 TSG\x03\x00\x00\x00\x01\x00\x00\x00\x03\x03\x0bA New Movie\x00\x00 TSG\x02\x00\x00\x00\x80\x00\x00\x00\x00\x14\x00\x00\x00\x00\x01\x00 TSG\x03\x00\x00\x00\x94\x00\x00\x00\x00}\x00\x00\x00\x00\x01\x00<\x00\x00\x00\x14\x00\x00\x00P\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00;\x00\x00\x00')

In [188]:
class GeneralGroup:
    def __init__(self):
        pass

    def ReadFromDataBlock(self, block, cfp):
        self.data1, self.data2 = FRead_VirtualGroup(block, cfp)

In [196]:
block = data[cfp.fpIndex:cfp.fpIndex+cfp.cbIndex]

gg = GeneralGroup()
gg.ReadFromDataBlock(block, cfp)

gg.data1
gg.data2

b'\x00\x00\x00\x00\x11\x01\x00\x00\x02\x08\x00\x00\x02\x00\x00\x00 TSG\x02\x00\x00\x00\x00\x00\x00\x00 TSG\x03\x00\x00\x00\x01\x00\x00\x00\x03\x03\x0bA New Movie\x00\x00 TSG\x02\x00\x00\x00\x80\x00\x00\x00\x00\x14\x00\x00\x00\x00\x01\x00 TSG\x03\x00\x00\x00\x94\x00\x00\x00\x00}\x00\x00\x00\x00\x01\x00<\x00\x00\x00\x14\x00\x00\x00P\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00;\x00\x00\x00'

In [ ]:
for (icrp = _pggcrp->IvMac(); icrp-- != 0;)
        {
            pcrpbg = (CRPBG *)_pggcrp->QvFixedGet(icrp, &cbVar);

In [197]:
# /***************************************************************************
#     Read a group from file and return it.
# ***************************************************************************/
# PGeneralGroup GeneralGroup::PggRead(PFIL pfil, FP fp, long cb, short *pbo, short *posk)
# {
#     DataBlock blck(pfil, fp, cb);
#     return PggRead(&blck, pbo, posk);
# }

def PggReadFromFile(data, fp, cb, cfp):
    block = data[fp:fp+cb]
    return PggReadFromDataBlock(block, cfp)
    
PggReadFromFile(data, cfp.fpIndex ,cfp.cbIndex, cfp)

(b' TSG\x03\x00\x00\x00\x94\x00\x00\x00\x00}\x00\x00\x00\x00\x01\x00',
 b'\x00\x00\x00\x00\x11\x01\x00\x00\x02\x08\x00\x00\x02\x00\x00\x00 TSG\x02\x00\x00\x00\x00\x00\x00\x00 TSG\x03\x00\x00\x00\x01\x00\x00\x00\x03\x03\x0bA New Movie\x00\x00 TSG\x02\x00\x00\x00\x80\x00\x00\x00\x00\x14\x00\x00\x00\x00\x01\x00 TSG\x03\x00\x00\x00\x94\x00\x00\x00\x00}\x00\x00\x00\x00\x01\x00<\x00\x00\x00\x14\x00\x00\x00P\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00;\x00\x00\x00')

In [193]:
# /***************************************************************************
#     Static method: open an existing file as a chunky file.  Increments the
#     open count.
# ***************************************************************************/
# PChunkyFile ChunkyFile::PcflOpen(Filename *pfni, ulong grfcfl)
# {
#     AssertPo(pfni, ffniFile);
#     PChunkyFile pcfl;
#     ulong grffil;

#     Assert(!(grfcfl & fcflTemp), "can't open a file as temp");
#     if (pvNil != (pcfl = PcflFromFni(pfni)))
#     {
#         if (!pcfl->FSetGrfcfl(grfcfl))
#             return pvNil;
#         pcfl->AddRef();
#         return pcfl;
#     }

#     grffil = _GrffilFromGrfcfl(grfcfl);
#     if ((pcfl = NewObj ChunkyFile()) == pvNil)
#         goto LFail;

#     if ((pcfl->_csto.pfil = FIL::PfilOpen(pfni, grffil)) == pvNil || !pcfl->_FReadIndex())
#     {
#         goto LFail;
#     }
#     if (tYes != pcfl->_TValidIndex())
#     {
#     LFail:
#         ReleasePpo(&pcfl);
#         PushErc(ercCflOpen);
#         return pvNil;
#     }
#     AssertDo(pcfl->FSetGrfcfl(grfcfl), 0);

#     // We don't assert with fcflGraph, because we've already
#     // called _TValidIndex.
#     AssertPo(pcfl, fcflFull);
#     return pcfl;
# }

In [194]:
# /***************************************************************************
#     Verifies that this is a chunky file and reads the index into memory.
#     Sets the _fpFreeMap and _cbFreeMap fields and sets the _fFreeMapNotRead
#     flag. The free map is read separately by _ReadFreeMap. This is to avoid
#     hogging memory for the free map when we may never need it.
# ***************************************************************************/
# bool ChunkyFile::_FReadIndex(void)
# {
#     AssertBaseThis(0);
#     AssertPo(_csto.pfil, 0);
#     Assert(!_fInvalidMainFile, 0);

#     ChunkyFilePrefix cfp;
#     FP fpMac;
#     short bo;
#     short osk;
#     long cbVar;
#     long cbRgch;
#     long icrp, ccrp;
#     long cbFixed;
#     ByteOrderMask bom;

#     // used for old name stuff
#     SZS szsName;
#     String stn;
#     bool fOldNames;

#     // used for old index stuff
#     bool fOldIndex;

#     Assert(_pggcrp == pvNil && _csto.pglfsm == pvNil && _cstoExtra.pfil == pvNil && _cstoExtra.pglfsm == pvNil,
#            "cfl has wrong non-nil entries");

#     // verify that this is a chunky file
#     if ((fpMac = _csto.pfil->FpMac()) < size(ChunkyFilePrefix))
#         return fFalse;

#     if (!_csto.pfil->FReadRgb(&cfp, size(cfp), 0))
#         return fFalse;

#     // check the magic number and byte order indicator
#     if (cfp.lwMagic != klwMagicChunky || cfp.bo != kboCur && cfp.bo != kboOther)
#     {
#         return fFalse;
#     }

#     if (cfp.bo == kboOther)
#         SwapBytesBom(&cfp, kbomCfp);

#     // check the version numbers
#     if (!cfp.dver.FReadable(kcvnCur, kcvnMin))
#         return fFalse;

#     // if the file has old style chunk names, translate them
#     fOldNames = cfp.dver._swCur < kcvnMinStnNames;

#     // whether the index needs converted
#     fOldIndex = cfp.dver._swCur < kcvnMinGrfcrp;

#     // verify the fp's and cb's
#     // the index and map should be last
#     if (!FIn(cfp.fpMac, size(cfp), fpMac + 1) || !FIn(cfp.fpIndex, size(cfp), cfp.fpMac + 1) ||
#         !FIn(cfp.cbIndex, 1, cfp.fpMac - cfp.fpIndex + 1) || cfp.fpMap != cfp.fpIndex + cfp.cbIndex ||
#         cfp.fpMap + cfp.cbMap != cfp.fpMac)
#     {
#         return fFalse;
#     }

#     // read and validate the index
#     if ((_pggcrp = GeneralGroup::PggRead(_csto.pfil, cfp.fpIndex, cfp.cbIndex, &bo, &osk)) == pvNil)
#     {
#         return fFalse;
#     }

#     cbFixed = _pggcrp->CbFixed();
#     if (cbFixed != size(CRPBG) && (fOldIndex || cbFixed != size(CRPSM)))
#         return fFalse;

#     // Clean the index
#     AssertBomRglw(kbomKid, size(ChildChunkIdentification));
#     _pggcrp->Lock();

#     if (cbFixed == size(CRPBG))
#     {
#         // Big index
#         CRPBG *pcrpbg;

#         bom = (bo != kboCur) ? (fOldIndex ? kbomCrpbgBytes : kbomCrpbgGrfcrp) : bomNil;

#         for (icrp = _pggcrp->IvMac(); icrp-- != 0;)
#         {
#             pcrpbg = (CRPBG *)_pggcrp->QvFixedGet(icrp, &cbVar);
# #ifndef CHUNK_BIG_INDEX
#             // make sure we can convert this CRP to a small index CRP
#             if (pcrpbg->cb > kcbMaxCrpsm || pcrpbg->ckid > kckidMax || pcrpbg->ccrpRef > kckidMax)
#             {
#                 goto LBadFile;
#             }
# #endif //! CHUNK_BIG_INDEX
#             if (!FIn(cbVar, 0, kcbMax))
#                 goto LBadFile;
#             if (bomNil != bom)
#                 SwapBytesBom(pcrpbg, bom);
#             if (!FIn(pcrpbg->ckid, 0, cbVar / size(ChildChunkIdentification) + 1) || (cbRgch = pcrpbg->CbRgch(cbVar)) > kcbMaxDataStn)
#             {
#                 goto LBadFile;
#             }
#             if (bomNil != bom && pcrpbg->ckid > 0)
#             {
#                 SwapBytesRglw(_pggcrp->QvGet(icrp), pcrpbg->ckid * (size(ChildChunkIdentification) / size(long)));
#             }
#             pcrpbg->rti = rtiNil;

#             if (fOldIndex)
#             {
#                 ulong grfcrp = fcrpNil;

#                 if (pcrpbg->fLoner)
#                     grfcrp |= fcrpLoner;
#                 if (pcrpbg->fPacked)
#                     grfcrp |= fcrpPacked;

#                 pcrpbg->grfcrp = grfcrp;
#             }
#             else
#                 pcrpbg->ClearGrfcrp(fcrpOnExtra);

#             if (pcrpbg->ccrpRef == 0)
#                 pcrpbg->SetGrfcrp(fcrpLoner);

#             AssertIn(cbRgch, 0, kcbMaxDataStn + 1);
#             if (fOldNames && cbRgch > 0)
#             {
#                 // translate the name
#                 long bvRgch = pcrpbg->BvRgch();

#                 if (cbRgch < size(szsName))
#                 {
#                     long cbNew;

#                     CopyPb(PvAddBv(_pggcrp->QvGet(icrp), bvRgch), szsName, cbRgch);
#                     szsName[cbRgch] = 0;
#                     stn.SetSzs(szsName);
#                     cbNew = stn.CbData();
#                     if (cbRgch != cbNew)
#                     {
#                         _pggcrp->Unlock();
#                         if (cbRgch < cbNew)
#                         {
#                             if (!_pggcrp->FInsertRgb(icrp, bvRgch, cbNew - cbRgch, pvNil))
#                             {
#                                 goto LNukeName;
#                             }
#                         }
#                         else
#                             _pggcrp->DeleteRgb(icrp, bvRgch, cbRgch - cbNew);
#                         _pggcrp->Lock();
#                     }
#                     stn.GetData(PvAddBv(_pggcrp->QvGet(icrp), bvRgch));
#                 }
#                 else
#                 {
#                     // just nuke the name
#                     _pggcrp->Unlock();
#                 LNukeName:
#                     _pggcrp->DeleteRgb(icrp, bvRgch, cbRgch);
#                     _pggcrp->Lock();
#                 }
#             }
#         }
#     }
#     else
#     {
#         // Small index
#         Assert(size(CRPSM) == cbFixed, 0);
#         CRPSM *pcrpsm;

#         bom = (bo != kboCur) ? kbomCrpsm : bomNil;
#         for (icrp = _pggcrp->IvMac(); icrp-- != 0;)
#         {
#             pcrpsm = (CRPSM *)_pggcrp->QvFixedGet(icrp, &cbVar);
#             if (!FIn(cbVar, 0, kcbMax))
#                 goto LBadFile;
#             if (bomNil != bom)
#                 SwapBytesBom(pcrpsm, bom);
#             if (!FIn(pcrpsm->ckid, 0, cbVar / size(ChildChunkIdentification) + 1) || (cbRgch = pcrpsm->CbRgch(cbVar)) > kcbMaxDataStn)
#             {
#                 goto LBadFile;
#             }
#             if (pcrpsm->ckid > 0 && bomNil != bom)
#             {
#                 SwapBytesRglw(_pggcrp->QvGet(icrp), pcrpsm->ckid * (size(ChildChunkIdentification) / size(long)));
#             }

#             pcrpsm->ClearGrfcrp(fcrpOnExtra);
#             if (pcrpsm->ccrpRef == 0 && !pcrpsm->Grfcrp(fcrpLoner))
#             {
#             LBadFile:
#                 _pggcrp->Unlock();
#                 Bug("corrupt crp");
#                 ReleasePpo(&_pggcrp);
#                 return fFalse;
#             }
#         }
#     }

#     if (size(CRP) != cbFixed)
#     {
#         // need to convert the index (from big to small or small to big)
#         PGeneralGroup pggcrp;
#         CRPOTH *pcrpOld;
#         CRP crp;

#         if (pvNil == (pggcrp = GeneralGroup::PggNew(size(CRP), _pggcrp->IvMac())))
#             goto LFail;

#         for (ccrp = _pggcrp->IvMac(), icrp = 0; icrp < ccrp; icrp++)
#         {
#             pcrpOld = (CRPOTH *)_pggcrp->QvFixedGet(icrp, &cbVar);
#             crp.cki = pcrpOld->cki;
#             crp.fp = pcrpOld->fp;
#             crp.SetCb(pcrpOld->Cb());
#             crp.ckid = (CKID)pcrpOld->ckid;
#             crp.ccrpRef = (CKID)pcrpOld->ccrpRef;
#             crp.AssignGrfcrp(pcrpOld->Grfcrp());
# #ifdef CHUNK_BIG_INDEX
#             crp.rti = rtiNil;
# #endif // CHUNK_BIG_INDEX

#             if (!pggcrp->FInsert(icrp, _pggcrp->Cb(icrp), _pggcrp->QvGet(icrp), &crp))
#             {
#                 ReleasePpo(&pggcrp);
#             LFail:
#                 _pggcrp->Unlock();
#                 return fFalse;
#             }
#         }
#         _pggcrp->Unlock();
#         ReleasePpo(&_pggcrp);

#         _pggcrp = pggcrp;
#     }
#     else
#         _pggcrp->Unlock();

#     _cbFreeMap = cfp.cbMap;
#     _fpFreeMap = cfp.fpMap;
#     _fFreeMapNotRead = cfp.cbMap > 0;

#     _csto.fpMac = cfp.fpIndex;
#     return fTrue;
# }

425